In [3]:
# ───────────────────────────────────────────────────────────────────────────────
# 1) Make sure we start in /content
%cd /content

# 2) Uninstall any existing transformers install
!pip uninstall -y transformers

# 3) Remove any leftover “transformers” directory
!rm -rf transformers

# 4) Clone the official Transformers repo
!git clone https://github.com/huggingface/transformers.git

# 5) Enter the repo and install in editable mode
%cd transformers
!pip install -e .

# 6) Install all other dependencies
!pip install -q accelerate datasets evaluate sentencepiece!=0.1.92 scikit-learn

# 7) Go to the GPT-2 CLM example folder and install example‐specific requirements
%cd examples/pytorch/language-modeling
!pip install -r requirements.txt

# 8) Verify GPU is available
!nvidia-smi

# 9) Launch GPT-2 fine-tuning on WikiText-2 using all available GPUs and mixed precision
!accelerate launch --mixed_precision fp16 run_clm.py \
  --model_name_or_path gpt2 \
  --dataset_name wikitext \
  --dataset_config_name wikitext-2-raw-v1 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --do_train \
  --do_eval \
  --fp16 \
  --output_dir /content/gpt2-wikitext-clm \
  --overwrite_output_dir \
  --num_train_epochs 2
# ───────────────────────────────────────────────────────────────────────────────

/content
Found existing installation: transformers 4.52.0.dev0
Uninstalling transformers-4.52.0.dev0:
  Successfully uninstalled transformers-4.52.0.dev0
Cloning into 'transformers'...
remote: Enumerating objects: 279325, done.
remote: Counting objects: 100% (625/625), done.
remote: Compressing objects: 100% (415/415), done.
remote: Total 279325 (delta 407), reused 215 (delta 205), pack-reused 278700 (from 4)
Receiving objects: 100% (279325/279325), 291.56 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (207306/207306), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-0.editable-py3-none-any.whl size=15244 sha256=98232eb0bcb36a4c7ca26f

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os

save_path = "/content/drive/MyDrive/fine_tuned_models/gpt2-wikitext"
os.makedirs(save_path, exist_ok=True)

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# 1) The real training output
trained_dir = "/content/gpt2-wikitext-clm"
assert os.path.isdir(trained_dir), f"{trained_dir} not found!"

# 2) Where you want to save on Drive
save_dir = "/content/drive/MyDrive/fine_tuned_models/my_finetuned_gpt2"
os.makedirs(save_dir, exist_ok=True)

# 3) Load locally (skip HF hub)
model = AutoModelForCausalLM.from_pretrained(trained_dir, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(trained_dir, local_files_only=True)

# 4) Save your final model & tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("✅ Saved to", save_dir)

✅ Saved to /content/drive/MyDrive/fine_tuned_models/my_finetuned_gpt2


In [12]:
# run in a Colab cell
!cp -r /content/gpt2-wikitext-clm /content/drive/MyDrive/fine_tuned_models/

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1) Your fine-tuned model folder on Drive
model_path = "/content/drive/MyDrive/fine_tuned_models/my_finetuned_gpt2"

# 2) Load tokenizer + model locally
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model     = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)

# 3) Move model to GPU if you have one
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 4) Tokenize input and also move tensors to the same device
inputs = tokenizer("Hello AI,", return_tensors="pt").to(device)

# 5) Generate
outputs = model.generate(**inputs, max_new_tokens=50)

# 6) Decode & print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello AI, the game 's protagonist , is a young man who has been abducted by a group of aliens . He is rescued by a group of aliens who have been sent to Earth to rescue him . 
 = = Plot = = 
 The game '


In [14]:
# after model is loaded
num_params = sum(p.numel() for p in model.parameters())
print(f"{num_params/1e6:.2f}M parameters")

124.44M parameters
